# Interaktive Folium-Karte: Dynamische Farbskalen und erweiterte Tooltips für Konstanz (AfD) und Kreuzlingen (SVP)

In diesem Abschnitt wird eine interaktive Karte mit Folium erstellt, die die AfD-Anteile in Konstanz und die SVP-Anteile in Kreuzlingen mit dynamischen Farbskalen und erweiterten Tooltips visualisiert. Es wird sichergestellt, dass alle benötigten Felder vorhanden sind und Fehler bei fehlenden Feldern abgefangen werden.

## 1. Vorbereitung der Daten für interaktive Kartenvisualisierung

Stelle sicher, dass alle benötigten Spalten in den GeoDataFrames `gdf_kn` (Konstanz) und `gdf_kr` (Kreuzlingen) vorhanden sind. Fehlende Felder werden ggf. aus vorhandenen Daten berechnet.

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os


# --- Daten laden ---
data_dir = "your_path"

# Konstanz: Shapefile und Wahldaten
shp_path_kn = "your_path/Konstanz/Kleinräumige_Gliederung_-5522761317040177735/Stadtteilfläche.shp"
csv_kn = f"{data_dir}/output_preprocessing/df_kn_wahlergebnis_demo.csv"
gdf_kn = gpd.read_file(shp_path_kn)
df_kn = pd.read_csv(csv_kn)


jahr_kn = df_kn['wahljahr'].max()
# Alle relevanten Altersgruppen für Konstanz mit aufnehmen!
altersgruppen_kn = [
    'einw_hw_unter18',
    'einw_hw_18_bis_unter_30',
    'einw_hw_30_bis_unter_40',
    'einw_hw_40_bis_unter_50',
    'einw_hw_50_bis_unter_60',
    'einw_hw_60_und_aelter'
]
df_kn_map = df_kn[df_kn['wahljahr'] == jahr_kn][
    ['stadtteil', 'AfD', 'einw_hw_gesamt', 'einw_hw_frauen', 'einw_hw_nichtdeutsch'] + altersgruppen_kn
]
gdf_kn['stadtteil'] = gdf_kn['STT_NAME'].str.strip()
# Korrigiere die Schreibweise im DataFrame für den Join
gdf_kn['stadtteil'] = gdf_kn['stadtteil'].str.replace('_', '-', regex=False).str.strip()
gdf_kn = gdf_kn.merge(df_kn_map, on='stadtteil', how='left')

# Kreuzlingen: GeoJSON und Wahldaten
geojson_kr = f"{data_dir}/Kreuzlingen:Kanton Thurgau/export-3.geojson"
csv_kr = f"{data_dir}/output_preprocessing/df_kreuzlingen_demographie_parteistaerke.csv"
gdf_kr = gpd.read_file(geojson_kr)
df_kr = pd.read_csv(csv_kr)

jahr_kr = df_kr['WAHLJAHR'].max()
# Alle Altersgruppen für Kreuzlingen mit aufnehmen!
df_kr_map = df_kr[df_kr['WAHLJAHR'] == jahr_kr][
    ['GEMEINDE_NAME', 'SVP', 'Männlich_Schweiz', 'Weiblich_Schweiz', 'Männlich_Ausland', 'Weiblich_Ausland']
]
gdf_kr['GEMEINDE_NAME'] = gdf_kr['name'].str.strip()
gdf_kr = gdf_kr.merge(df_kr_map, on='GEMEINDE_NAME', how='left')

/Users/tessabartels/Desktop/Coding/Datenjournalismus/venv/lib/python3.10/site-packages/pyogrio/raw.py:198: RuntimeWarning: /Users/tessabartels/Desktop/Coding/Datenjournalismus/data/Konstanz/Kleinräumige_Gliederung_-5522761317040177735/Stadtteilfläche.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np

# Annahme: gdf_kn und gdf_kr sind bereits geladen und enthalten die Geometrien

# --- Konstanz: Benötigte Felder prüfen und ggf. berechnen ---
kn_required_fields = ['stadtteil', 'AfD', 'einw_hw_männer', 'einw_hw_frauen', 'einw_hw_nichtdeutsch']
for field in kn_required_fields:
    if field not in gdf_kn.columns:
        if field == 'einw_hw_männer':
            # Berechne Männer aus Gesamt - Frauen, falls möglich
            if 'einw_hw_gesamt' in gdf_kn.columns and 'einw_hw_frauen' in gdf_kn.columns:
                gdf_kn['einw_hw_männer'] = gdf_kn['einw_hw_gesamt'] - gdf_kn['einw_hw_frauen']
            else:
                gdf_kn['einw_hw_männer'] = np.nan
        else:
            gdf_kn[field] = np.nan

# --- Kreuzlingen: Benötigte Felder prüfen und ggf. berechnen ---
kr_required_fields = ['GEMEINDE_NAME', 'SVP', 'Männlich_Schweiz', 'Weiblich_Schweiz', 'Ausland_gesamt']
for field in kr_required_fields:
    if field not in gdf_kr.columns:
        if field == 'Ausland_gesamt':
            # Berechne Ausland_gesamt aus Männlich_Ausland + Weiblich_Ausland, falls möglich
            if 'Männlich_Ausland' in gdf_kr.columns and 'Weiblich_Ausland' in gdf_kr.columns:
                gdf_kr['Ausland_gesamt'] = gdf_kr['Männlich_Ausland'] + gdf_kr['Weiblich_Ausland']
            else:
                gdf_kr['Ausland_gesamt'] = np.nan
        else:
            gdf_kr[field] = np.nan

## 2. Definition und Anwendung dynamischer Farbskalen mit Folium und Branca

Berechne die Minimal- und Maximalwerte für AfD und SVP und erstelle entsprechende Farbskalen mit `branca.colormap.linear`. Diese werden später an die jeweiligen Layer gebunden.

In [3]:
import folium
import branca

# Farbskalen für AfD (Konstanz) und SVP (Kreuzlingen)
afd_min, afd_max = gdf_kn['AfD'].min(), gdf_kn['AfD'].max()
svp_min, svp_max = gdf_kr['SVP'].min(), gdf_kr['SVP'].max()

afd_colormap = branca.colormap.linear.Reds_09.scale(afd_min, afd_max)
svp_colormap = branca.colormap.linear.Blues_09.scale(svp_min, svp_max)

## 3. Erstellung des Konstanz-Layers (AfD) mit erweiterten Tooltips

Füge `gdf_kn` als GeoJson-Layer hinzu. Die `style_function` sorgt für die dynamische Farbgebung. Der Tooltip zeigt alle gewünschten Felder an, sofern sie existieren.

In [4]:
# Konstanz: Prozentuale Anteile berechnen und auf 1 Nachkommastellen runden
if 'einw_hw_gesamt' in gdf_kn.columns:
    gdf_kn['Männer (%)'] = (100 * gdf_kn['einw_hw_männer'] / gdf_kn['einw_hw_gesamt']).round(1)
    gdf_kn['Frauen (%)'] = (100 * gdf_kn['einw_hw_frauen'] / gdf_kn['einw_hw_gesamt']).round(1)
    gdf_kn['Nichtdeutsche (%)'] = (100 * gdf_kn['einw_hw_nichtdeutsch'] / gdf_kn['einw_hw_gesamt']).round(1)

In [5]:
# Lade die vorberechneten Durchschnittsalter pro Gemeinde
df_avg_alter_kn = pd.read_csv("/Users/tessabartels/Desktop/Coding/Datenjournalismus/data/output_preprocessing/df_avg_alter_kn.csv")

# Mergen mit gdf_kr (ersetzt die bisherige Berechnung)
gdf_kn = gdf_kn.merge(df_avg_alter_kn, on='stadtteil', how='left')


In [6]:
gdf_kn['Durchschnittsalter']

0     38.0
1     40.3
2     46.6
3     46.6
4     44.4
5     45.5
6     37.5
7     38.8
8     45.6
9     44.8
10    46.3
11    41.4
12    40.6
13    41.0
14    40.9
Name: Durchschnittsalter, dtype: float64

In [7]:
# Felder und Aliase für den Tooltip (nur vorhandene Felder verwenden)


kn_tooltip_fields = ['stadtteil', 'AfD', 'Männer (%)', 'Frauen (%)', 'Nichtdeutsche (%)', 'Durchschnittsalter']
kn_tooltip_aliases = ['Stadtteil', 'AfD-Anteil (%)', 'Männer (%)', 'Frauen (%)', 'Nichtdeutsche (%)', 'Durchschnittsalter']

# Karte initialisieren
m = folium.Map(location=[47.66, 9.18], zoom_start=12)

# Konstanz-Layer
folium.GeoJson(
    gdf_kn,
    name="Konstanz AfD",
    style_function=lambda feature: {
        "fillColor": afd_colormap(feature["properties"].get("AfD")) if feature["properties"].get("AfD") is not None else "#cccccc",
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.7,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=kn_tooltip_fields,
        aliases=kn_tooltip_aliases,
        localize=True
    )
).add_to(m)
afd_colormap.caption = "AfD-Anteil (%)"
afd_colormap.add_to(m)

## 4. Erstellung des Kreuzlingen-Layers (SVP) mit erweiterten Tooltips

Füge `gdf_kr` als GeoJson-Layer hinzu. Die `style_function` sorgt für die dynamische Farbgebung. Der Tooltip zeigt alle gewünschten Felder an, sofern sie existieren.

In [8]:
# Durchschnittsalter für Kreuzlingen-Gemeinden direkt aus CSV einlesen und mergen

# Lade die vorberechneten Durchschnittsalter pro Gemeinde
df_avg_alter_kr = pd.read_csv("/Users/tessabartels/Desktop/Coding/Datenjournalismus/data/output_preprocessing/df_avg_alter_kr.csv")
df_avg_alter_kr = df_avg_alter_kr.rename(columns={'Altersdurchschnitt': 'Durchschnittsalter'})


# Mergen mit gdf_kr (ersetzt die bisherige Berechnung)
gdf_kr = gdf_kr.merge(df_avg_alter_kr, on='GEMEINDE_NAME', how='left')


In [9]:
# Kreuzlingen: Prozentuale Anteile berechnen und auf 1 Nachkommastellen runden
if 'Männlich_Schweiz' in gdf_kr.columns and 'Weiblich_Schweiz' in gdf_kr.columns and 'Ausland_gesamt' in gdf_kr.columns:
    gdf_kr['Gesamt'] = gdf_kr['Männlich_Schweiz'] + gdf_kr['Weiblich_Schweiz'] + gdf_kr['Ausland_gesamt']
    gdf_kr['Männer (%)'] = (100 * gdf_kr['Männlich_Schweiz'] / gdf_kr['Gesamt']).round(1)
    gdf_kr['Frauen (%)'] = (100 * gdf_kr['Weiblich_Schweiz'] / gdf_kr['Gesamt']).round(1)
    gdf_kr['Nicht-Schweizer (%)'] = (100 * gdf_kr['Ausland_gesamt'] / gdf_kr['Gesamt']).round(1)

In [10]:
# Felder und Aliase für den Tooltip (nur vorhandene Felder verwenden)

kr_tooltip_fields = ['GEMEINDE_NAME', 'SVP', 'Männer (%)', 'Frauen (%)', 'Nicht-Schweizer (%)', 'Durchschnittsalter']
kr_tooltip_aliases = ['Gemeinde', 'SVP-Anteil (%)', 'Männer (%)', 'Frauen (%)', 'Nicht-Schweizer (%)', 'Durchschnittsalter']
# Kreuzlingen-Layer
folium.GeoJson(
    gdf_kr,
    name="Kreuzlingen SVP",
    style_function=lambda feature: {
        "fillColor": svp_colormap(feature["properties"].get("SVP")) if feature["properties"].get("SVP") is not None else "#cccccc",
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.7,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=kr_tooltip_fields,
        aliases=kr_tooltip_aliases,
        localize=True
    )
).add_to(m)
svp_colormap.caption = "SVP-Anteil (%)"
svp_colormap.add_to(m)

## 5. Fehlerbehandlung: Prüfung auf fehlende Felder in GeoDataFrames

Vor dem Hinzufügen der Layer wird geprüft, ob alle für den Tooltip gewünschten Felder in den GeoDataFrames vorhanden sind. Falls Felder fehlen, wird eine Warnung ausgegeben und die Tooltips werden entsprechend angepasst.

In [11]:
# Warnungen für fehlende Felder ausgeben
missing_kn = [f for f in ['stadtteil', 'AfD', 'einw_hw_männer', 'einw_hw_frauen', 'einw_hw_nichtdeutsch'] if f not in gdf_kn.columns]
missing_kr = [f for f in ['GEMEINDE_NAME', 'SVP', 'Männlich_Schweiz', 'Weiblich_Schweiz', 'Ausland_gesamt'] if f not in gdf_kr.columns]

if missing_kn:
    print("Warnung: Folgende Felder fehlen in gdf_kn und werden im Tooltip nicht angezeigt:", missing_kn)
if missing_kr:
    print("Warnung: Folgende Felder fehlen in gdf_kr und werden im Tooltip nicht angezeigt:", missing_kr)

## 6. Hinzufügen von LayerControl und Anzeige/Speicherung der Karte

Füge die LayerControl hinzu, zeige die Karte im Notebook an und speichere sie optional als HTML-Datei.

In [12]:


folium.LayerControl().add_to(m)



m  # Zeigt die Karte im Notebook an



In [13]:
# Optional: Karte als HTML speichern
m.save("karte_konstanz_kreuzlingen.html")
print("Karte wurde als 'karte_konstanz_kreuzlingen.html' gespeichert.")

Karte wurde als 'karte_konstanz_kreuzlingen.html' gespeichert.
